# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [ ]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

## Experiments with embedding models



*   "dunzhang/stella_en_400M_v5"
*   "dunzhang/stella_en_1.5B_v5"
*   'Alibaba-NLP/gte-large-en-v1.5'
*   "Snowflake/snowflake-arctic-embed-l"
*   "jinaai/jina-embeddings-v3"



In [ ]:
import os
import datasets
import datetime
import re
import pandas as pd
import faiss
import numpy as np

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads

In [ ]:
from google.colab import userdata, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### MedQA-USMLE-4-options dataset

In [ ]:
# run experiments on train split
data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
data = data.add_column('original_quest_id', range(len(data)))

In [ ]:
# sample questions for experiments
sampled_data = data.shuffle(seed=42).select(range(50))

In [ ]:
sampled_data[0]

{'question': 'A 35-year-old woman comes to your office with a variety of complaints. As part of her evaluation, she undergoes laboratory testing which reveals the presence of anti-centromere antibodies. All of the following symptoms and signs would be expected to be present EXCEPT:',
 'answer': 'Hypercoagulable state',
 'options': {'A': 'Pallor, cyanosis, and erythema of the hands',
  'B': 'Blanching vascular abnormalities',
  'C': 'Hypercoagulable state',
  'D': 'Heartburn and regurgitation'},
 'meta_info': 'step1',
 'answer_idx': 'C',
 'metamap_phrases': ['35 year old woman',
  'office',
  'variety',
  'complaints',
  'part of',
  'evaluation',
  'laboratory testing',
  'reveals',
  'presence',
  'anti-centromere antibodies',
  'following symptoms',
  'signs',
  'expected to',
  'present']}

### MedQA dataset

In [ ]:
# download MedQA data from https://drive.google.com/file/d/1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw/view?usp=sharing
!gdown -q 1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw
!unzip -q data_clean.zip -d /content/medQA

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/response.py", line 748, in _error_catcher
    yield
  File "/usr/local/lib/python3.10/dist-packages/urllib3/response.py", line 873, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
  File "/usr/local/lib/python3.10/dist-packages/urllib3/response.py", line 856, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/usr/lib/python3.10/http/client.py", line 466, in read
    s = self.fp.read(amt)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10/ssl.py", line 1159, in read
    return self._sslobj.read(len, buffer)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/gdown

### Document embedding and indexing

In [ ]:
def load_documents():
    '''
    Loads reference textbooks for MedQA as a list of Document objects.
    '''

    medqa_path = '/content/medQA/data_clean/textbooks/en/'
    loader = DirectoryLoader(medqa_path, glob='**/*.txt', loader_cls=TextLoader)

    # list of 'Document' files, with 'source'/file path as metadata
    documents = loader.load()

    return documents

In [ ]:
import pickle

In [ ]:
def create_vector_store(list_documents, text_splitter, embed_model, save_to_file=True, file_path='drive/MyDrive/ia024/projeto/entrega4/retriever/', add_info=None):
    '''
    Splits and embeds a list of documents.
    '''

    # splits list of documents into chunks and adds the source txt filename as metadata
    split_documents = text_splitter.create_documents([doc.page_content for doc in list_documents], metadatas=[{'ref': doc.metadata['source'].split('/')[-1]} for doc in list_documents])
    print('Documents created.')

    # embeds and indexes each chunk using FAISS
    vector_store = FAISS.from_documents(split_documents, embed_model)

    filename = file_path + 'faiss__' + add_info + '__' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    with open(filename, 'wb') as f:
      pickle.dump(vector_store, f)

    if save_to_file:
        filename = file_path + 'faiss__' + add_info + '__' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        vector_store.save_local(filename)

    return vector_store

In [ ]:
# textbooks loaded as documents
docs = load_documents()

In [ ]:
# chunk embedding model
# emb_model_name = 'sentence-transformers/all-MiniLM-L6-v2'

# test different embedding models
emb_model_name = [
    # "dunzhang/stella_en_400M_v5",
    # 'Alibaba-NLP/gte-base-en-v1.5',
    "Snowflake/snowflake-arctic-embed-m",
    # "jinaai/jina-embeddings-v3"
]

encode_kwargs = {'batch_size': 32}
model_kwargs = {'trust_remote_code': True}


In [ ]:
# embedding_model = HuggingFaceEmbeddings(model_name=emb_model_name[0], model_kwargs={'trust_remote_code': True})
embedding_model = HuggingFaceEmbeddings(model_name=emb_model_name[2], model_kwargs=model_kwargs, encode_kwargs=encode_kwargs, show_progress = True)

<ipython-input-13-6726f208bd06>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=emb_model_name[2], model_kwargs=model_kwargs, encode_kwargs=encode_kwargs, show_progress = True)


#### Recursive character text splitter

Analysis of paragraph length shows that, with exception of one textbook (Surgery_Schwartz), every other book has an average length of up to 600 characters. We'll try a variation of this window size of 300. The overlap size will be fixed at 10% of the window size.

In [ ]:
for chunk_size, chunk_overlap in zip([600], [60]):

    print(chunk_size, chunk_overlap)

    # create text splitter with specified chunk and overlap sizes
    rec_text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # create a vector store from that splitter
    tmp_faiss = create_vector_store(docs, rec_text_splitter, embedding_model, add_info=f'{chunk_size}_{chunk_overlap}')

600 60
Documents created.


Batches:   0%|          | 0/6348 [00:00<?, ?it/s]

### ReAct agent

In [ ]:
model = 'gpt-4o-mini'
model_temp = 0.5

In [ ]:
# react prompt
react_prompt_template = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action
Observation: the output of the action.
... (this Thought/Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt = ChatPromptTemplate.from_template(react_prompt_template)

In [ ]:
llm = ChatOpenAI(
        model=model,
        temperature=model_temp,
    )

In [ ]:
retrievers = [
    '/content/drive/MyDrive/ia024/projeto/entrega4/retriever/faiss__600_60__20241110_203300',
]

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name=emb_model_name[2], model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

<ipython-input-16-38375d0a49d6>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=emb_model_name[2], model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/84.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
for retriever in retrievers:

    print(retriever)

    # load saved faiss indexes
    faiss = FAISS.load_local(
        folder_path=retriever,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )

    # create a search tool from the faiss retriever
    tool_search_faiss = create_retriever_tool(
        retriever=faiss.as_retriever(search_kwargs={'k': 3}),
        name='search_docs',
        description='Searchs the query in documents and returns the top-3 most relevant ones.',
    )

    # create agent with tools
    agent = create_react_agent(
        llm=llm,
        tools=[tool_search_faiss],
        prompt=react_prompt
    )

    agent_executor = AgentExecutor(
        agent=agent,
        tools=[tool_search_faiss],
        verbose=False,
        handle_parsing_errors=True,
        return_intermediate_steps=True,
        max_iterations=5
    )

    tmp = retriever.split('/')[-1].split('__')[1]
    filename = f'/content/drive/MyDrive/ia024/projeto/entrega4/results/answers_faiss_{tmp}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'

    # run agent
    with open(filename, 'w') as f:

        f.write(''.center(10, '-'))
        f.write('\n')
        f.write(f'Model: {model}\n')
        f.write(f'Temperature: {model_temp}\n')
        f.write(f'Prompt: {react_prompt}\n')
        f.write(''.center(10, '-'))
        f.write('\n[{')

        for n, question in enumerate(tqdm(sampled_data)):

            res = agent_executor.invoke({'question': question['question'], 'options': question['options']})
            f.write(f'"{n}": ')

            # serialize results as a string
            f.write(dumps(res))
            f.write(',')

        f.write('"placeholder": "placeholder"') # add a placeholder k:v pair for trailling comma above
        f.write('}]')

/content/drive/MyDrive/ia024/projeto/entrega4/retriever/faiss__600_60__20241110_203300


100%|██████████| 50/50 [06:08<00:00,  7.38s/it]


#### Results with FAISS in 50 sample questions

In [ ]:
result_logs = [
    '/content/drive/MyDrive/ia024/projeto/entrega4/results/answers_faiss_600_60_20241110_205729.txt',
]

In [ ]:
acc_df = pd.DataFrame(columns=['Chunk Size', 'Overlap Size', 'Accuracy'])

In [ ]:
for logfile in result_logs:

    chunk_size = logfile.split('/')[-1].split('_')[2]
    chunk_overlap = logfile.split('/')[-1].split('_')[3]

    print(chunk_size, chunk_overlap)

    with open(logfile) as f:

        # load log file with LLM results
        tmp = f.readlines()

        # recover results as a dictionary
        recovered_results = loads(tmp[-1])[0]

        limit_error = list()
        not_parsed = list()
        answers = list()

        # matches literals A, B, C, or D at the beginning of a sentence (followed by the text of the corresponding option)
        # e.g. A (answer text) => A
        # other cases will be ignored
        ans_ptn = re.compile(r'(^[A|B|C|D])\s\(')

        for n, (k, v) in enumerate(recovered_results.items()):

            # skip placeholder key
            if k == 'placeholder':
                continue

            # max_iteration set to 5
            if v['output'] == 'Agent stopped due to iteration limit or time limit.':
                answers.append(None)
                limit_error.append(k)

            elif len(v['output']) > 1:
                tmp_output = ans_ptn.findall(v['output'])
                if len(tmp_output) == 1:
                    answers.append(tmp_output[0])
                elif len(tmp_output) > 1:
                    answers.append(None)
                    not_parsed.append(k)
                    print(tmp_output)
                else:
                    answers.append(None)
                    not_parsed.append(k)
                    print(k, v['output'])

            else:
                if v['output'] in ['A', 'B', 'C', 'D']:
                    answers.append(v['output'])
                else:
                    answers.append(None)

    # unanswered questions or answers not properly parsed
    print('Unanswered questions or answers not properly parsed: ', len(limit_error))

    # parsing errors/other errors
    print('Parsing errors/other errors: ', len(not_parsed))

    # accuracy
    accuracy = sum([pred == gt for gt, pred in zip(sampled_data['answer_idx'], answers)]) / len(sampled_data)
    print('Accuracy: ', accuracy)
    print('\n')

    acc_df.loc[len(acc_df)] = [chunk_size, chunk_overlap, accuracy]

600 60
Unanswered questions or answers not properly parsed:  6
Parsing errors/other errors:  0
Accuracy:  0.66




<ipython-input-24-944509deb1cf>:14: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  recovered_results = loads(tmp[-1])[0]


In [ ]:
acc_df

,Chunk Size,Overlap Size,Accuracy
0,600,60,0.66


### Retrieved context evaluation with RAGAS

In [ ]:
# result_dict[question]['intermediate_steps'] -> List[List[AgentAction(tool, tool_input, log), response: str]]
def format_context(answer):

    formatted_output = ''
    for i in answer['intermediate_steps']:
        formatted_output += i[1]

    return formatted_output

In [ ]:
class RelevantSentence(BaseModel):
    '''A sentence you believe is relevant to answer the provided question.
    '''
    sentence: str

class Context(BaseModel):
    '''The list of contexts you retrieved based on a question.
    '''
    contexts: List[RelevantSentence]

In [ ]:
ext_sentence_prompt = '''
Please extract relevant sentences from the provided context that can potentially help answer the following question.
If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phrase "Insufficient Information".
While extracting candidate sentences you’re not allowed to make any changes to sentences from given context.
Here is the context: {context}
And here is the question: {question}
'''.strip()

ext_sentence_prompt_template = ChatPromptTemplate.from_template(ext_sentence_prompt)

In [ ]:
chain_extraction = ext_sentence_prompt_template | llm.with_structured_output(Context)

In [ ]:
ctxt_rel_df = pd.DataFrame(columns=['Chunk Size', 'Overlap Size', 'Context relevancy'])

In [ ]:
for logfile in result_logs:

    chunk_size = logfile.split('/')[-1].split('_')[2]
    chunk_overlap = logfile.split('/')[-1].split('_')[3]

    print(chunk_size, chunk_overlap)

    # load log file with LLM results
    with open(logfile) as f:
        tmp = f.readlines()

    # recover results as a dictionary
    recovered_results = loads(tmp[-1])[0]

    # format recovered contexts as a list of strings
    react_logs = list()
    for i in recovered_results.values():
        if i == 'placeholder':
            continue
        react_logs.append(format_context(i))

    sampled_data_2 = sampled_data.add_column('answer_react_faiss', answers)
    sampled_data_2 = sampled_data_2.add_column('react_log', react_logs)

    relevant_contexts = list()
    context_rel = list()

    for question in tqdm(sampled_data_2):

        try:
            # select relevant sentences
            contexts = chain_extraction.invoke({'context': question['react_log'], 'question': question['question']})
            relevant_contexts.append(contexts)

            # fraction of relevant sentences in the context
            context_rel.append(len(contexts.contexts) / len(question['react_log'].split('.')))
        except:
            relevant_contexts.append(None)
            context_rel.append(np.nan)

    cont_rel_dataset = np.nanmean(context_rel)
    ctxt_rel_df.loc[len(ctxt_rel_df)] = [chunk_size, chunk_overlap, cont_rel_dataset]


600 60


100%|██████████| 50/50 [01:16<00:00,  1.54s/it]


In [ ]:
ctxt_rel_df

,Chunk Size,Overlap Size,Context relevancy
0,600,60,1.493333
